In [40]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from dotenv import load_dotenv

load_dotenv()

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-notebook")

from utils import get_dataset_from_pickle

import plotly.express as px
px.set_mapbox_access_token(os.getenv('MAPBOX_TOKEN'))

# City of Los Angeles: Proposing a Strategy for Optimizing Parking Enforcement Deployment
**Author: Evan Gabrielson**

### Overview: 
---
In recent years, the City of Los Angeles has faced a significant financial challenge with its parking enforcement operations. Although traffic fines once provided a steady stream of revenue, a troubling shift occurred starting in 2017. Since then, the costs associated with salaries, equipment, and other expenses for parking enforcement have skyrocketed to over $809 million, while the revenue generated from parking ticket fines has lagged behind at $617 million. This $192 million shortfall highlights the urgent need for more efficient and effective strategies in managing parking violations and enforcement.

### Business Understanding:
---
In his book titled "The High Cost of Free Parking", LA resident and UCLA professor Donald Shoup outlines the necessity of parking enforcement policy to "curb" the excess time and resources a free parking state begets. 
Shoup presents two key recommendations for reform to improve parking policy: pricing curb parking according to fair market value and redistributing parking revenue to neighborhoods for community investment.
Parking enforcement is only necessary to the extent that it improves the maintenance and fair distribution of parking resources such that all actors can benefit equally from public road infrastructure.
Today, however, LA Department of Transportation (LADOT) policy makers are forced to counteract skyhigh salary expenses with parking violation fees well above fair market value while reinvesting nothing back into the communities. 
Until LADOT can produce a net profit from parking enforcement, the citizenry of Los Angeles must continue to expect rising parking violation fees and zero community reinvestment.

In this project, I propose several data-driven strategies for optimizing parking enforcer deployment which LADOT can employ to close the gap between revenue and payroll. 
Here are some facts about LADOT as it functions today:
- LADOT currently deploys an equal distribution of enforcement officers across the City of LA. 
- Enforcers are on duty 24/7
- Parking citations fees range from $53 - $350

In [33]:
citations_df = get_dataset_from_pickle('data/pickle/citations_v0_bronze.pickle')
citations_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20529058 entries, 2022-12-13 16:20:58.225000 to 2023-12-27 10:07:08.455000
Data columns (total 5 columns):
 #   Column          Dtype   
---  ------          -----   
 0   fine_amount     float32 
 1   agency          category
 2   violation_code  category
 3   loc_lat         float32 
 4   loc_long        float32 
dtypes: category(2), float32(3)
memory usage: 450.3 MB


### Identifying Outliers in Data
Outlier analysis in exploratory data analysis is crucial because it helps identify data points that deviate significantly from the rest, which may indicate errors, unusual events, or important insights. Understanding these outliers allows for more accurate modeling, better decision-making, and can prevent misleading conclusions.

When working with geospatial data, it is often a good idea to cluster coordinates using algorithmic approaches like KMeans and Agglomerative Clustering or, if available, overlay predefined regions that would be relevant to the model. While geospatial data may be useful for plotting, clusters are better for identifying patterns through machine learning and statistical analysis. A built-in perk of overlaying our geospatial data over the City of LA Parking Enforcement District map is that we automatically filter out data points that lie outside a valid district region. We'll use `geopandas` to facilitate the spatial join operation used to group rows by district.

In [34]:
# Recreate citations DataFrame as GeoDataFrame
citations_gdf = gpd.GeoDataFrame(citations_df, geometry=gpd.points_from_xy(citations_df['loc_long'], citations_df['loc_lat']))

# Build districts GeoDataFrame and set Coordinate Reference System (CRS)
districts_gdf = gpd.read_file("data/geojson/parking_enforcement_districts.geojson")
citations_gdf.set_crs(districts_gdf.crs, inplace=True)

# Compute spatial join (sjoin) operation on 'District' GeoJSON attribute
citations_joined_gdf = gpd.sjoin(citations_gdf, districts_gdf[['geometry', 'District']], how='left', predicate='within')

# Store 'District' label in new 'district' column
citations_df['district'] = citations_joined_gdf['District'].astype('category')
citations_df['district'].value_counts()

district
Hollywood    5025700
Western      4595190
Central      4467880
Valley       3809492
Southern     2556676
Name: count, dtype: int64

We can now view the number of data points lying outside the predefined Parking Enforcement Districts and store a new pickle file with the `loc_lat` and `loc_long` columns then remove them to save memory space for future processing.

In [39]:
# Find centroids for each district
centroids = citations_df.groupby('district')[['loc_lat', 'loc_long']].mean().reset_index()
centroids.head()

,district,loc_lat,loc_long
0,Central,34.057568,-118.240814
1,Hollywood,34.084881,-118.320679
2,Southern,33.962658,-118.289276
3,Valley,34.191124,-118.454765
4,Western,34.029087,-118.424713


In [35]:
# Plot geographic outliers

for district in centroids:
    plt.plot(district)
citations_df['district'].isna().sum()

np.int64(74120)

### Understanding Trends in Violations over Time

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

In [ ]:
violations_df = pd.read_pickle('data/pickle/violations_v0_clean.pickle')
violations_df.info()

In [ ]:
violations_per_year = violations_df.resample('YE').count()
print(violations_per_year)

In [ ]:
# Plot number of violations per year since 2000
plt.figure(figsize=(20,5))
sns.barplot(x=list(violations_per_year.index.year), y=violations_per_year['violation_code'])
plt.title('Number of Violations per Year (2014-2023)', fontdict={'fontsize':22})
plt.ylabel("Number of Violations (Millions)",fontdict={'fontsize':16})
plt.xlabel("Year", fontdict={'fontsize':16})
plt.savefig('./images/violations_per_year.png')

In [ ]:
# Plot seasonality decomposition
timeseries = violations_df['violation_code'].resample('d').count().loc['2023-01-05':'2023-06-05']
decomposition = seasonal_decompose(timeseries)
trend, seasonal, residuals = decomposition.trend, decomposition.seasonal, decomposition.resid

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(411)
plt.title('Seasonality Decomposition for 2024')
plt.plot(timeseries,label='Original',color='blue')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend,label='Trend',color='green')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal,label='Seasonal',color='orange')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(residuals,label='Residuals',color='red')
plt.legend(loc='upper left')

TODO - Add plotly interactive map of violation count by region in representative date range (hexmap?)

#### Temporal Analysis: Weekly & Hourly Basis
TODO - Add findings

In [ ]:
# separate violations into weekdays and weekends
weekdays = violations_df[violations_df.index.weekday<5]
weekends = violations_df[violations_df.index.weekday>=5]

In [ ]:
plt.figure(figsize=(15,8))
(weekdays.groupby(weekdays.index.hour).size()/10/52/5).plot(kind='area',label='Total Weekday Violations',linewidth=2,alpha=.5)
(weekends.groupby(weekends.index.hour).size()/10/52/2).plot(kind = 'area',label='Total Weekend Violations',linewidth=2,alpha=.5)
plt.legend()
plt.title('Number of Violations per Hour',fontdict={'fontsize':22})
plt.xlabel('Time Of Day (H)',fontdict={'fontsize':18})
plt.ylabel('Violations per Week',fontdict={'fontsize':18})
plt.savefig('./images/hm_weekday_vs_weekend.png')

#### Temporal Analysis: Daily Basis
TODO - Add findings

In [ ]:
# Weekday analysis
weekdays_by_hour = violations_df.groupby([violations_df.index.hour,violations_df.index.weekday]).size()

plt.figure(figsize=(12,12))
sns.heatmap(weekdays_by_hour.unstack(), xticklabels = ['Mon','Tues','Wed','Thurs','Fri','Sat','Sun'], linewidths=1)
plt.title('Violations by Day of Week',fontdict={'fontsize':22})
plt.ylabel('Hour',fontdict={'fontsize':16})
plt.xlabel('Day of Week',fontdict={'fontsize':16})
plt.savefig('./images/hm_daily.png')

#### Spatial Analysis: Geographic Basis
TODO - Add findings

In [ ]:
import plotly.express as px

In [ ]:
# Get violation counts grouped by latitude and longitude
BIN_THRESHOLD = 0.02
to_bin = lambda x: np.floor(x / BIN_THRESHOLD) * BIN_THRESHOLD
violations_df['latbin'] = to_bin(violations_df['loc_lat'])
violations_df['longbin'] = to_bin(violations_df['loc_long'])

geographically_clustered_violations = violations_df.groupby([
    'latbin', 
    'longbin'
]).size().reset_index(name="Count")

In [ ]:
# TODO - [SECURITY RISK] Use .env token instead


In [ ]:
fig = px.scatter_mapbox(geographically_clustered_violations, lat='latbin', lon='longbin', size="Count", color="Count", color_continuous_scale=px.colors.cyclical.IceFire)
fig.update_layout(height=512, width=512)
fig.show()

TODO - Based on population density

#### Categorical Analysis: Violation Codes
TODO - Add findings

| Code | Description |
| ---- | ----------- |
| 80.69BS | PARKING PROHIBITED |
| 88.13B+ | FAILURE TO PAY FOR A PARKING METER SPACE |
| 80.56E4 | PARKED IN NO STOPPING CURB ZONE |
| 80.58L | PARKED IN PREFERENTIAL PARKING DISTRICT WITHOUT PERMIT |
| 5204A- | REGISTRATION TAB IMPROPERLY ATTACHED TO LICENSE |
| 80.69B | PARKING PROHIBITED |
| 5200 | LICENSE PLATE DISPLAY ISSUE |
| 80.69C | TIME LIMIT EXCEEDED |
| 80.69AP+ | CAR WITH TRAILER PARKING PROHIBITED | 
| 80.56 | PARKED IN PASSENGER LOADING ZONE |
| 80.61 | PARKED IN ALLEY |
| 22500E | BLOCKING DRIVEWAY |
| 80.70 | PARKED IN ANTI-GRIDLOCK ZONE |
| 80.69AA+ | PARKING PROHIBITED | 

In [ ]:
INCLUDE_THRESHOLD = violations_df.shape[0]*1//100
print(f"A given violation code must exceed a count of {INCLUDE_THRESHOLD} to be included in this analysis.")
violation_code_counts = violations_df['violation_code'].value_counts()
filtered_codes = violation_code_counts[violation_code_counts > INCLUDE_THRESHOLD].index
violation_code_counts = violations_df[violations_df['violation_code'].isin(filtered_codes)]

In [ ]:
plt.figure(figsize=(20, 5))
sns.countplot(data=violation_code_counts, y='violation_code', order=filtered_codes)
plt.title('Number of Violations by Code (2014-2024)', fontdict={'fontsize':22})
plt.ylabel("Violation Code", fontdict={'fontsize':16})
plt.xlabel("Number of Violations (Millions)",fontdict={'fontsize':16})
plt.savefig('./images/violations_by_code.png')